In [1]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, warnings, random, datetime

from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold, GroupKFold, TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm


import math
warnings.filterwarnings('ignore')

In [2]:
########################### Helpers
#################################################################################
## Seeder
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [3]:
########################### Vars
#################################################################################
SEED = 42
seed_everything(SEED)
LOCAL_TEST = False
TARGET = 'isFraud'
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')

In [4]:
features_columns = ['TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', 'V109', 'V110', 'V111', 'V112', 'V113', 'V114', 'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 'V121', 'V122', 'V123', 'V124', 'V125', 'V126', 'V127', 'V128', 'V129', 'V130', 'V131', 'V132', 'V133', 'V134', 'V135', 'V136', 'V137', 'V138', 'V139', 'V140', 'V141', 'V142', 'V143', 'V144', 'V145', 'V146', 'V147', 'V148', 'V149', 'V150', 'V151', 'V152', 'V153', 'V154', 'V155', 'V156', 'V157', 'V158', 'V159', 'V160', 'V161', 'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V168', 'V169', 'V170', 'V171', 'V172', 'V173', 'V174', 'V175', 'V176', 'V177', 'V178', 'V179', 'V180', 'V181', 'V182', 'V183', 'V184', 'V185', 'V186', 'V187', 'V188', 'V189', 'V190', 'V191', 'V192', 'V193', 'V194', 'V195', 'V196', 'V197', 'V198', 'V199', 'V200', 'V201', 'V202', 'V203', 'V204', 'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V211', 'V212', 'V213', 'V214', 'V215', 'V216', 'V217', 'V218', 'V219', 'V220', 'V221', 'V222', 'V223', 'V224', 'V225', 'V226', 'V227', 'V228', 'V229', 'V230', 'V231', 'V232', 'V233', 'V234', 'V235', 'V236', 'V237', 'V238', 'V239', 'V240', 'V241', 'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V248', 'V249', 'V250', 'V251', 'V252', 'V253', 'V254', 'V255', 'V256', 'V257', 'V258', 'V259', 'V260', 'V261', 'V262', 'V263', 'V264', 'V265', 'V266', 'V267', 'V268', 'V269', 'V270', 'V271', 'V272', 'V273', 'V274', 'V275', 'V276', 'V277', 'V278', 'V279', 'V280', 'V281', 'V282', 'V283', 'V284', 'V285', 'V286', 'V287', 'V288', 'V289', 'V290', 'V291', 'V292', 'V293', 'V294', 'V295', 'V296', 'V297', 'V298', 'V299', 'V300', 'V301', 'V302', 'V303', 'V304', 'V305', 'V306', 'V307', 'V308', 'V309', 'V310', 'V311', 'V312', 'V313', 'V314', 'V315', 'V316', 'V317', 'V318', 'V319', 'V320', 'V321', 'V322', 'V323', 'V324', 'V325', 'V326', 'V327', 'V328', 'V329', 'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338', 'V339', 'TransactionAmt_check', 'card1_TransactionAmt_mean', 'card1_TransactionAmt_std', 'card1_TransactionAmt_max', 'card1_TransactionAmt_min', 'card1_TransactionAmt_median', 'card1_TransactionAmt_nunique', 'card1_TransactionAmt_skew', 'card2_TransactionAmt_mean', 'card2_TransactionAmt_std', 'card2_TransactionAmt_max', 'card2_TransactionAmt_min', 'card2_TransactionAmt_median', 'card2_TransactionAmt_nunique', 'card2_TransactionAmt_skew', 'card3_TransactionAmt_mean', 'card3_TransactionAmt_std', 'card3_TransactionAmt_max', 'card3_TransactionAmt_min', 'card3_TransactionAmt_median', 'card3_TransactionAmt_nunique', 'card3_TransactionAmt_skew', 'card5_TransactionAmt_mean', 'card5_TransactionAmt_std', 'card5_TransactionAmt_max', 'card5_TransactionAmt_min', 'card5_TransactionAmt_median', 'card5_TransactionAmt_nunique', 'card5_TransactionAmt_skew', 'uid_TransactionAmt_mean', 'uid_TransactionAmt_std', 'uid_TransactionAmt_max', 'uid_TransactionAmt_min', 'uid_TransactionAmt_median', 'uid_TransactionAmt_nunique', 'uid_TransactionAmt_skew', 'uid2_TransactionAmt_mean', 'uid2_TransactionAmt_std', 'uid2_TransactionAmt_max', 'uid2_TransactionAmt_min', 'uid2_TransactionAmt_median', 'uid2_TransactionAmt_nunique', 'uid2_TransactionAmt_skew', 'uid3_TransactionAmt_mean', 'uid3_TransactionAmt_std', 'uid3_TransactionAmt_max', 'uid3_TransactionAmt_min', 'uid3_TransactionAmt_median', 'uid3_TransactionAmt_nunique', 'uid3_TransactionAmt_skew', 'uid4_TransactionAmt_mean', 'uid4_TransactionAmt_std', 'uid4_TransactionAmt_max', 'uid4_TransactionAmt_min', 'uid4_TransactionAmt_median', 'uid4_TransactionAmt_nunique', 'uid4_TransactionAmt_skew', 'uid5_TransactionAmt_mean', 'uid5_TransactionAmt_std', 'uid5_TransactionAmt_max', 'uid5_TransactionAmt_min', 'uid5_TransactionAmt_median', 'uid5_TransactionAmt_nunique', 'uid5_TransactionAmt_skew', 'addr1_TransactionAmt_mean', 'addr1_TransactionAmt_std', 'addr1_TransactionAmt_max', 'addr1_TransactionAmt_min', 'addr1_TransactionAmt_median', 'addr1_TransactionAmt_nunique', 'addr1_TransactionAmt_skew', 'uid7_TransactionAmt_mean', 'uid7_TransactionAmt_std', 'uid7_TransactionAmt_max', 'uid7_TransactionAmt_min', 'uid7_TransactionAmt_median', 'uid7_TransactionAmt_nunique', 'uid7_TransactionAmt_skew', 'uid8_TransactionAmt_mean', 'uid8_TransactionAmt_std', 'uid8_TransactionAmt_min', 'uid8_TransactionAmt_median', 'uid8_TransactionAmt_skew', 'card1_D15_mean', 'card1_D15_std', 'card1_D15_max', 'card1_D15_min', 'card1_D15_median', 'card1_D15_nunique', 'card1_D15_skew', 'card2_D15_mean', 'card2_D15_std', 'card2_D15_max', 'card2_D15_min', 'card2_D15_median', 'card2_D15_nunique', 'card2_D15_skew', 'card3_D15_mean', 'card3_D15_std', 'card3_D15_max', 'card3_D15_min', 'card3_D15_median', 'card3_D15_nunique', 'card3_D15_skew', 'card5_D15_mean', 'card5_D15_std', 'card5_D15_max', 'card5_D15_min', 'card5_D15_median', 'card5_D15_nunique', 'card5_D15_skew', 'uid_D15_mean', 'uid_D15_std', 'uid_D15_max', 'uid_D15_min', 'uid_D15_median', 'uid_D15_nunique', 'uid_D15_skew', 'uid2_D15_mean', 'uid2_D15_std', 'uid2_D15_max', 'uid2_D15_min', 'uid2_D15_median', 'uid2_D15_nunique', 'uid2_D15_skew', 'uid3_D15_mean', 'uid3_D15_std', 'uid3_D15_max', 'uid3_D15_min', 'uid3_D15_median', 'uid3_D15_nunique', 'uid3_D15_skew', 'uid4_D15_std', 'uid4_D15_max', 'uid4_D15_min', 'uid4_D15_nunique', 'uid4_D15_skew', 'uid5_D15_std', 'uid5_D15_skew', 'addr1_D15_mean', 'addr1_D15_std', 'addr1_D15_max', 'addr1_D15_min', 'addr1_D15_median', 'addr1_D15_nunique', 'addr1_D15_skew', 'uid7_D15_skew', 'uid8_D15_skew', 'card1_dist1_mean', 'card1_dist1_std', 'card1_dist1_max', 'card1_dist1_min', 'card1_dist1_median', 'card1_dist1_nunique', 'card1_dist1_skew', 'card2_dist1_mean', 'card2_dist1_std', 'card2_dist1_max', 'card2_dist1_min', 'card2_dist1_median', 'card2_dist1_nunique', 'card2_dist1_skew', 'card3_dist1_mean', 'card3_dist1_std', 'card3_dist1_max', 'card3_dist1_min', 'card3_dist1_median', 'card3_dist1_nunique', 'card3_dist1_skew', 'card5_dist1_mean', 'card5_dist1_std', 'card5_dist1_max', 'card5_dist1_min', 'card5_dist1_median', 'card5_dist1_nunique', 'card5_dist1_skew', 'uid_dist1_mean', 'uid_dist1_std', 'uid_dist1_max', 'uid_dist1_min', 'uid_dist1_median', 'uid_dist1_nunique', 'uid_dist1_skew', 'uid2_dist1_mean', 'uid2_dist1_std', 'uid2_dist1_max', 'uid2_dist1_min', 'uid2_dist1_median', 'uid2_dist1_nunique', 'uid2_dist1_skew', 'uid3_dist1_mean', 'uid3_dist1_std', 'uid3_dist1_max', 'uid3_dist1_min', 'uid3_dist1_median', 'uid3_dist1_nunique', 'uid3_dist1_skew', 'uid4_dist1_mean', 'uid4_dist1_std', 'uid4_dist1_max', 'uid4_dist1_min', 'uid4_dist1_median', 'uid4_dist1_nunique', 'uid4_dist1_skew', 'uid5_dist1_mean', 'uid5_dist1_std', 'uid5_dist1_max', 'uid5_dist1_min', 'uid5_dist1_median', 'uid5_dist1_nunique', 'uid5_dist1_skew', 'addr1_dist1_mean', 'addr1_dist1_std', 'addr1_dist1_max', 'addr1_dist1_min', 'addr1_dist1_median', 'addr1_dist1_nunique', 'addr1_dist1_skew', 'uid7_dist1_std', 'card1_TransactionAmt_cumsum', 'card1_TransactionAmt_cumcount', 'card2_TransactionAmt_cumsum', 'card2_TransactionAmt_cumcount', 'card3_TransactionAmt_cumsum', 'card3_TransactionAmt_cumcount', 'card5_TransactionAmt_cumsum', 'card5_TransactionAmt_cumcount', 'uid_TransactionAmt_cumsum', 'uid_TransactionAmt_cumcount', 'uid2_TransactionAmt_cumsum', 'uid2_TransactionAmt_cumcount', 'uid3_TransactionAmt_cumsum', 'uid3_TransactionAmt_cumcount', 'uid4_TransactionAmt_cumsum', 'uid4_TransactionAmt_cumcount', 'uid5_TransactionAmt_cumsum', 'uid5_TransactionAmt_cumcount', 'addr1_TransactionAmt_cumsum', 'addr1_TransactionAmt_cumcount', 'uid7_TransactionAmt_cumsum', 'uid7_TransactionAmt_cumcount', 'uid8_TransactionAmt_cumsum', 'uid8_TransactionAmt_cumcount', 'email_check', 'R_emaildomain_prefix2', 'P_emaildomain_prefix2', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_32', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo', 'id_33_0', 'id_33_1', 'is_na_sum2', 'not_na_sum2', 'DeviceInfo_device', 'id_30_device', 'UserAgent', 'ParsingError', 'card1_fq_enc', 'card2_fq_enc', 'card3_fq_enc', 'card5_fq_enc', 'C1_fq_enc', 'C2_fq_enc', 'C3_fq_enc', 'C4_fq_enc', 'C5_fq_enc', 'C6_fq_enc', 'C7_fq_enc', 'C8_fq_enc', 'C9_fq_enc', 'C10_fq_enc', 'C11_fq_enc', 'C13_fq_enc', 'C14_fq_enc', 'D1_fq_enc', 'D2_fq_enc', 'D8_fq_enc', 'addr1_fq_enc', 'addr2_fq_enc', 'dist1_fq_enc', 'dist2_fq_enc', 'P_emaildomain_fq_enc', 'R_emaildomain_fq_enc', 'R_emaildomain_prefix2_fq_enc', 'P_emaildomain_prefix2_fq_enc', 'DeviceInfo_fq_enc', 'DeviceInfo_device_fq_enc', 'DeviceInfo_version_fq_enc', 'id_30_fq_enc', 'id_30_device_fq_enc', 'id_30_version_fq_enc', 'id_31_device_fq_enc', 'id_33_fq_enc', 'uid_fq_enc', 'uid2_fq_enc', 'uid3_fq_enc', 'uid4_fq_enc', 'uid5_fq_enc', 'uid6_fq_enc', 'uid7_fq_enc', 'uid8_fq_enc', 'uid_DT_M']
print('loading training data...')
train_df = pd.read_pickle(r'data\train_final.pkl')
print('loading testing data...')
test_df = pd.read_pickle(r'data\test_final.pkl')

loading training data...
loading testing data...


In [5]:
########################### Export
def make_submission(train_df, test_df, features_columns, target, rf_params,round_ = 2200, NFOLDS=2):

    folds = GroupKFold(n_splits=NFOLDS)

    X,y = train_df[features_columns], train_df[TARGET]    
    P,P_y = test_df[features_columns], test_df[TARGET]
    print(X.columns)
    #y[:] = 1
    
    split_groups = train_df['DT_M']
    # We don't need original sets anymore
    # let's reduce it
    train_df = train_df[['TransactionID',TARGET]] 
    test_df = test_df[['TransactionID',TARGET]] 
    test_df['prediction'] = 0
    gc.collect()
    
    oof = np.zeros(len(train_df))
    predictions = np.zeros(len(test_df))

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y, groups=split_groups)):
        print('Fold:',fold_)
        estimator = RandomForestClassifier(**rf_params)        
        estimator.fit(X.iloc[trn_idx,:],y[trn_idx])

        oof_preds = estimator.predict_proba(X.iloc[val_idx,:])[:,1]
        oof[val_idx] = (oof_preds - oof_preds.min())/(oof_preds.max() - oof_preds.min())
        test_df['prediction'] += estimator.predict_proba(P)[:,1]/NFOLDS
        
        del estimator
        gc.collect()

    print('OOF AUC:', metrics.roc_auc_score(y, oof))       
        
    #return metrics.roc_auc_score(y, oof),test_df
    return test_df

In [6]:
########################### Model params
rf_params = {
             'max_depth': None,
             'max_features': 0.2,
             'min_samples_leaf': 20,
             'n_estimators': 1200,
             #'n_estimators': 5,
             'n_jobs': -1,
             'oob_score': True,
             'random_state': 42,
             'verbose': 100
             }

In [7]:
%%time
test_predictions = make_submission(train_df, test_df, features_columns, TARGET, rf_params,round_ = 2200, NFOLDS=6)
test_predictions['isFraud'] = test_predictions['prediction']
test_predictions[['TransactionID','isFraud']].to_csv(r'middle_result\test_rf_0924.csv', index=False)

Index(['TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4',
       'card5', 'card6', 'addr1', 'addr2',
       ...
       'id_33_fq_enc', 'uid_fq_enc', 'uid2_fq_enc', 'uid3_fq_enc',
       'uid4_fq_enc', 'uid5_fq_enc', 'uid6_fq_enc', 'uid7_fq_enc',
       'uid8_fq_enc', 'uid_DT_M'],
      dtype='object', length=727)
Fold: 0
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
building tree 1 of 1200building tree 4 of 1200building tree 2 of 1200building tree 3 of 1200building tree 5 of 1200building tree 6 of 1200building tree 7 of 1200building tree 8 of 1200building tree 9 of 1200building tree 10 of 1200building tree 11 of 1200building tree 12 of 1200











building tree 13 of 1200[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.3min

building tree 14 of 1200[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.3min

building tree 15 of 1200[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.3min

building tree 16 of 1200[Pa

building tree 99 of 1200[Parallel(n_jobs=-1)]: Done  87 tasks      | elapsed: 11.7min

[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed: 11.7min
building tree 100 of 1200
building tree 101 of 1200[Parallel(n_jobs=-1)]: Done  89 tasks      | elapsed: 11.7min

building tree 102 of 1200[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 11.7min

building tree 103 of 1200[Parallel(n_jobs=-1)]: Done  91 tasks      | elapsed: 11.9min

building tree 104 of 1200[Parallel(n_jobs=-1)]: Done  92 tasks      | elapsed: 11.9min

building tree 105 of 1200[Parallel(n_jobs=-1)]: Done  93 tasks      | elapsed: 12.1min

building tree 106 of 1200[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed: 12.1min

[Parallel(n_jobs=-1)]: Done  95 tasks      | elapsed: 12.3min
building tree 107 of 1200
building tree 108 of 1200[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed: 12.3min

building tree 109 of 1200[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed: 12.9min

building tree 110 of 1200[Paralle


building tree 193 of 1200[Parallel(n_jobs=-1)]: Done 181 tasks      | elapsed: 22.7min

building tree 194 of 1200[Parallel(n_jobs=-1)]: Done 182 tasks      | elapsed: 22.8min

building tree 195 of 1200[Parallel(n_jobs=-1)]: Done 183 tasks      | elapsed: 22.9min

building tree 196 of 1200[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 23.3min

building tree 197 of 1200[Parallel(n_jobs=-1)]: Done 185 tasks      | elapsed: 23.4min

[Parallel(n_jobs=-1)]: Done 186 tasks      | elapsed: 23.4min
building tree 198 of 1200
[Parallel(n_jobs=-1)]: Done 187 tasks      | elapsed: 23.4min
building tree 199 of 1200
building tree 200 of 1200[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed: 23.5min

building tree 201 of 1200[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed: 23.7min

building tree 202 of 1200[Parallel(n_jobs=-1)]: Done 190 tasks      | elapsed: 23.8min

building tree 203 of 1200[Parallel(n_jobs=-1)]: Done 191 tasks      | elapsed: 23.9min

building tree 204 of 1200[Paral


building tree 287 of 1200[Parallel(n_jobs=-1)]: Done 275 tasks      | elapsed: 34.0min

[Parallel(n_jobs=-1)]: Done 276 tasks      | elapsed: 34.0min
building tree 288 of 1200
building tree 289 of 1200[Parallel(n_jobs=-1)]: Done 277 tasks      | elapsed: 34.1min

building tree 290 of 1200[Parallel(n_jobs=-1)]: Done 278 tasks      | elapsed: 34.2min

building tree 291 of 1200[Parallel(n_jobs=-1)]: Done 279 tasks      | elapsed: 34.6min

building tree 292 of 1200[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed: 34.7min

building tree 293 of 1200[Parallel(n_jobs=-1)]: Done 281 tasks      | elapsed: 34.9min

building tree 294 of 1200[Parallel(n_jobs=-1)]: Done 282 tasks      | elapsed: 34.9min

building tree 295 of 1200[Parallel(n_jobs=-1)]: Done 283 tasks      | elapsed: 34.9min

building tree 296 of 1200[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed: 35.1min

building tree 297 of 1200[Parallel(n_jobs=-1)]: Done 285 tasks      | elapsed: 35.1min

building tree 298 of 1200[Paral


building tree 474 of 1200[Parallel(n_jobs=-1)]: Done 462 tasks      | elapsed: 56.8min

building tree 475 of 1200[Parallel(n_jobs=-1)]: Done 463 tasks      | elapsed: 56.9min

building tree 476 of 1200[Parallel(n_jobs=-1)]: Done 464 tasks      | elapsed: 57.0min

building tree 477 of 1200[Parallel(n_jobs=-1)]: Done 465 tasks      | elapsed: 57.0min

building tree 478 of 1200[Parallel(n_jobs=-1)]: Done 466 tasks      | elapsed: 57.1min

[Parallel(n_jobs=-1)]: Done 467 tasks      | elapsed: 57.2min
building tree 479 of 1200
building tree 480 of 1200[Parallel(n_jobs=-1)]: Done 468 tasks      | elapsed: 57.6min

building tree 481 of 1200[Parallel(n_jobs=-1)]: Done 469 tasks      | elapsed: 57.9min

building tree 482 of 1200[Parallel(n_jobs=-1)]: Done 470 tasks      | elapsed: 58.0min

building tree 483 of 1200[Parallel(n_jobs=-1)]: Done 471 tasks      | elapsed: 58.0min

building tree 484 of 1200[Parallel(n_jobs=-1)]: Done 472 tasks      | elapsed: 58.1min

building tree 485 of 1200[Paral

...

[Parallel(n_jobs=12)]: Done 594 tasks      | elapsed:   17.4s
[Parallel(n_jobs=12)]: Done 595 tasks      | elapsed:   17.4s
[Parallel(n_jobs=12)]: Done 596 tasks      | elapsed:   17.4s
[Parallel(n_jobs=12)]: Done 597 tasks      | elapsed:   17.4s
[Parallel(n_jobs=12)]: Done 598 tasks      | elapsed:   17.5s
[Parallel(n_jobs=12)]: Done 599 tasks      | elapsed:   17.5s
[Parallel(n_jobs=12)]: Done 600 tasks      | elapsed:   17.5s
[Parallel(n_jobs=12)]: Done 601 tasks      | elapsed:   17.6s
[Parallel(n_jobs=12)]: Done 602 tasks      | elapsed:   17.6s
[Parallel(n_jobs=12)]: Done 603 tasks      | elapsed:   17.7s
[Parallel(n_jobs=12)]: Done 604 tasks      | elapsed:   17.7s
[Parallel(n_jobs=12)]: Done 605 tasks      | elapsed:   17.7s
[Parallel(n_jobs=12)]: Done 606 tasks      | elapsed:   17.7s
[Parallel(n_jobs=12)]: Done 607 tasks      | elapsed:   17.8s
[Parallel(n_jobs=12)]: Done 608 tasks      | elapsed:   17.8s
[Parallel(n_jobs=12)]: Done 609 tasks      | elapsed:   17.8s
[Paralle

[Parallel(n_jobs=12)]: Done 727 tasks      | elapsed:   21.3s
[Parallel(n_jobs=12)]: Done 728 tasks      | elapsed:   21.3s
[Parallel(n_jobs=12)]: Done 729 tasks      | elapsed:   21.4s
[Parallel(n_jobs=12)]: Done 730 tasks      | elapsed:   21.4s
[Parallel(n_jobs=12)]: Done 731 tasks      | elapsed:   21.4s
[Parallel(n_jobs=12)]: Done 732 tasks      | elapsed:   21.4s
[Parallel(n_jobs=12)]: Done 733 tasks      | elapsed:   21.5s
[Parallel(n_jobs=12)]: Done 734 tasks      | elapsed:   21.5s
[Parallel(n_jobs=12)]: Done 735 tasks      | elapsed:   21.5s
[Parallel(n_jobs=12)]: Done 736 tasks      | elapsed:   21.5s
[Parallel(n_jobs=12)]: Done 737 tasks      | elapsed:   21.5s
[Parallel(n_jobs=12)]: Done 738 tasks      | elapsed:   21.6s
[Parallel(n_jobs=12)]: Done 739 tasks      | elapsed:   21.6s
[Parallel(n_jobs=12)]: Done 740 tasks      | elapsed:   21.7s
[Parallel(n_jobs=12)]: Done 741 tasks      | elapsed:   21.7s
[Parallel(n_jobs=12)]: Done 742 tasks      | elapsed:   21.8s
[Paralle

[Parallel(n_jobs=12)]: Done 860 tasks      | elapsed:   25.2s
[Parallel(n_jobs=12)]: Done 861 tasks      | elapsed:   25.2s
[Parallel(n_jobs=12)]: Done 862 tasks      | elapsed:   25.2s
[Parallel(n_jobs=12)]: Done 863 tasks      | elapsed:   25.2s
[Parallel(n_jobs=12)]: Done 864 tasks      | elapsed:   25.3s
[Parallel(n_jobs=12)]: Done 865 tasks      | elapsed:   25.3s
[Parallel(n_jobs=12)]: Done 866 tasks      | elapsed:   25.3s
[Parallel(n_jobs=12)]: Done 867 tasks      | elapsed:   25.3s
[Parallel(n_jobs=12)]: Done 868 tasks      | elapsed:   25.4s
[Parallel(n_jobs=12)]: Done 869 tasks      | elapsed:   25.4s
[Parallel(n_jobs=12)]: Done 870 tasks      | elapsed:   25.5s
[Parallel(n_jobs=12)]: Done 871 tasks      | elapsed:   25.5s
[Parallel(n_jobs=12)]: Done 872 tasks      | elapsed:   25.6s
[Parallel(n_jobs=12)]: Done 873 tasks      | elapsed:   25.6s
[Parallel(n_jobs=12)]: Done 874 tasks      | elapsed:   25.6s
[Parallel(n_jobs=12)]: Done 875 tasks      | elapsed:   25.6s
[Paralle

[Parallel(n_jobs=12)]: Done 993 tasks      | elapsed:   29.1s
[Parallel(n_jobs=12)]: Done 994 tasks      | elapsed:   29.2s
[Parallel(n_jobs=12)]: Done 995 tasks      | elapsed:   29.2s
[Parallel(n_jobs=12)]: Done 996 tasks      | elapsed:   29.2s
[Parallel(n_jobs=12)]: Done 997 tasks      | elapsed:   29.3s
[Parallel(n_jobs=12)]: Done 998 tasks      | elapsed:   29.3s
[Parallel(n_jobs=12)]: Done 999 tasks      | elapsed:   29.3s
[Parallel(n_jobs=12)]: Done 1000 tasks      | elapsed:   29.3s
[Parallel(n_jobs=12)]: Done 1001 tasks      | elapsed:   29.4s
[Parallel(n_jobs=12)]: Done 1002 tasks      | elapsed:   29.4s
[Parallel(n_jobs=12)]: Done 1003 tasks      | elapsed:   29.4s
[Parallel(n_jobs=12)]: Done 1004 tasks      | elapsed:   29.4s
[Parallel(n_jobs=12)]: Done 1005 tasks      | elapsed:   29.4s
[Parallel(n_jobs=12)]: Done 1006 tasks      | elapsed:   29.5s
[Parallel(n_jobs=12)]: Done 1007 tasks      | elapsed:   29.5s
[Parallel(n_jobs=12)]: Done 1008 tasks      | elapsed:   29.5s

[Parallel(n_jobs=12)]: Done 1124 tasks      | elapsed:   32.9s
[Parallel(n_jobs=12)]: Done 1125 tasks      | elapsed:   33.0s
[Parallel(n_jobs=12)]: Done 1126 tasks      | elapsed:   33.0s
[Parallel(n_jobs=12)]: Done 1127 tasks      | elapsed:   33.0s
[Parallel(n_jobs=12)]: Done 1128 tasks      | elapsed:   33.1s
[Parallel(n_jobs=12)]: Done 1129 tasks      | elapsed:   33.1s
[Parallel(n_jobs=12)]: Done 1130 tasks      | elapsed:   33.1s
[Parallel(n_jobs=12)]: Done 1131 tasks      | elapsed:   33.2s
[Parallel(n_jobs=12)]: Done 1132 tasks      | elapsed:   33.2s
[Parallel(n_jobs=12)]: Done 1133 tasks      | elapsed:   33.2s
[Parallel(n_jobs=12)]: Done 1134 tasks      | elapsed:   33.2s
[Parallel(n_jobs=12)]: Done 1135 tasks      | elapsed:   33.2s
[Parallel(n_jobs=12)]: Done 1136 tasks      | elapsed:   33.3s
[Parallel(n_jobs=12)]: Done 1137 tasks      | elapsed:   33.4s
[Parallel(n_jobs=12)]: Done 1138 tasks      | elapsed:   33.4s
[Parallel(n_jobs=12)]: Done 1139 tasks      | elapsed: 

In [8]:
#result

In [9]:
test_predictions

,TransactionID,isFraud,prediction
0,3663549,0.012445,0.012445
1,3663550,0.017466,0.017466
2,3663551,0.024061,0.024061
3,3663552,0.005779,0.005779
4,3663553,0.014670,0.014670
5,3663554,0.015650,0.015650
6,3663555,0.038390,0.038390
7,3663556,0.056355,0.056355
8,3663557,0.009966,0.009966
9,3663558,0.017390,0.017390


In [10]:
#help(CatBoostClassifier().fit)

In [11]:
#cat_params

In [12]:
#result

In [13]:
#cat_params

In [14]:
#prediction['learning_rate'][0.05]['isFraud'] = prediction['learning_rate'][0.05]['prediction']

In [15]:
#out.drop(['prediction'],axis = 1,inplace = True)

In [16]:
#out.to_csv(r'result\test_cat.csv',index = False)